# ¿Qué es EventBridge Scheduler?

Amazon EventBridge Scheduler es un programador sin servidor que le permite crear, ejecutar y administrar tareas desde un servicio central y administrado. Altamente escalable, EventBridge Scheduler le permite programar millones de tareas que pueden invocar más de 270 servicios de AWS y más de 6.000 operaciones de API. Sin necesidad de aprovisionar y administrar la infraestructura, ni de integrarse con varios servicios, EventBridge Scheduler le ofrece la posibilidad de realizar programaciones a escala y reducir los costes de mantenimiento.

EventBridge Scheduler entrega sus tareas de forma fiable, con mecanismos incorporados que ajustan sus programaciones en función de la disponibilidad de los objetivos descendentes. Con EventBridge Scheduler, puede crear programaciones utilizando expresiones cron y rate para patrones recurrentes, o configurar invocaciones únicas. Puede establecer ventanas de tiempo flexibles para la entrega, definir los límites de reintento y establecer el tiempo máximo de retención para los activadores fallidos. EventBridge Scheduler también le permite crear programaciones que se ejecutan en un horario específico o en un momento determinado, como el inicio de un evento o la finalización de un proceso.

In [ ]:
#!pip install botocore --upgrade pip
#!pip install boto3 --upgrade pip

In [1]:
import boto3
import logging
import json
from botocore.exceptions import ClientError
print('Boto3 version: {}'.format(boto3.__version__))

Boto3 version: 1.26.16


### Programación de un horario único

In [2]:
client = boto3.client('scheduler', region_name='us-east-1')
sns = boto3.client('sns', region_name='us-east-1')
iam = boto3.client('iam', region_name='us-east-1')

In [3]:
# Create a topic
topic_name = 'EventBridgeScheduler'
response = sns.create_topic(
    Name = topic_name
)
topic_arn = response['TopicArn']
while topic_arn == 'pending confirmation':
    response = sns.create_topic(
        Name = topic_name
    )

In [11]:
# Create a emaiil susbcription
email_address = "estebanpbuday@yahoo.es"
response = sns.subscribe(
    TopicArn = topic_arn,
    Protocol = 'email',
    Endpoint = email_address
)   

In [5]:
# Create a role

role_name = 'EventBridgeScheduler'
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "EventBridgeScheduler",
            "Effect": "Allow",
            "Principal": {
                "Service": "scheduler.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
            }
        ]
    }
role_description = 'EventBridge Scheduler Role'
tags = [
    {
        'Key': 'Environment',
        'Value': 'Production'
    },
    {
        'Key': 'Name',
        'Value': 'EventBridgeScheduler'
    }
]

In [6]:
response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
    Description=role_description,
    Tags=tags
)
role_arn = response['Role']['Arn']

In [15]:
# Create Schedule

event = client.create_schedule(
    Name='SendEmailOnce',
    ScheduleExpression='rate(15 minutes)',
    ScheduleExpressionTimezone='America/Argentina/Buenos_Aires',
    FlexibleTimeWindow={
            'Mode':'OFF'
        },
    Target={
            'Arn': topic_arn,
            'RoleArn': role_arn
        }
) 
    